In [52]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

SCROLL_PAUSE_TIME = 3
URL = 'https://www.stussy.com/collections/all-womens'
#URL = 'https://www.stussy.com/collections/all-mens'
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source,'html.parser')

In [53]:
product_url = []
picture_url = []
name = []
price = []
sales_price = []

timestamp = []
brand = []
product_type = []

color = []

for i in soup.find_all('li',{'class':'collection__product'}):
    
    price.append(None)
    sales_price.append(i.find('span',{'class':'product-card__price'}).text)
    
    timestamp.append(date_today)
    brand.append('Stüssy')
    product_type.append("Women's Clothing")
    
    item_color = []
    for color_i in i.find('ul',{'class':'product-card__thumbs'}).find_all('li',{'class':'product-card__thumb'}):
        index_color = str(color_i.img.get('alt')).find(' - ')
        item_color.append(str(color_i.img.get('alt'))[index_color+3:])
    color.append(item_color)
    name.append(i.find('a',{'class':'product-card__title'}).text)
    product_url.append("https://www.stussy.com" + i.find('a',{'class':'product-card__title'}).get('href'))
    index = str(i.find('div',{'class':'ratio-box'}).img.get('data-srcset')).find('180w,')
    picture_url.append("https:" + str(i.find('div',{'class':'ratio-box'}).img.get('data-srcset'))[:index].replace('180x','600x'))

In [54]:
product_description = []
material = []
picture_name = []
counter = 1

for item in product_url:
    
    ourUrl = urllib.request.urlopen(item)
    soup = BeautifulSoup(ourUrl,'html.parser')
    picture_name.append('stüssy-women_' + str(counter) + '.jpg')
   
    material_index = str(soup.find('div',{'class':'product__details rte'}).text).find('Imported')
    product_description.append(str(soup.find('div',{'class':'product__details rte'}).text)[:material_index].strip())
    material.append(None)
    counter += 1

In [55]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Stüssy,Women's Clothing,Stüssy / Nike Air Force 1 Low,SOLD OUTALL SALES FINALStüssy and Nike return ...,None,$130.00,"[Fossil/Fossil, Black/Black]",None,https://www.stussy.com/collections/all-womens/...,https://cdn.shopify.com/s/files/1/0087/6193/39...,2020-12-11,stüssy-women_1.jpg
1,Stüssy,Women's Clothing,Stüssy / Nike Force 1 (Youth),SOLD OUTALL SALES FINAL Stüssy and Nike return...,None,$75.00,"[Black/Black, Fossil/Fossil]",None,https://www.stussy.com/collections/all-womens/...,https://cdn.shopify.com/s/files/1/0087/6193/39...,2020-12-11,stüssy-women_2.jpg
2,Stüssy,Women's Clothing,Stüssy / Nike Force 1 (Toddler),SOLD OUTALL SALES FINAL Stüssy and Nike return...,None,$65.00,"[Fossil/Fossil, Black/Black]",None,https://www.stussy.com/collections/all-womens/...,https://cdn.shopify.com/s/files/1/0087/6193/39...,2020-12-11,stüssy-women_3.jpg
3,Stüssy,Women's Clothing,Stüssy / Nike Benassi Slide,SOLD OUT ALL SALES FINALStüssy and Nike retur...,None,$35.00,"[Black/Sail, Pine Green/Sail, Sail/Black]",None,https://www.stussy.com/collections/all-womens/...,https://cdn.shopify.com/s/files/1/0087/6193/39...,2020-12-11,stüssy-women_4.jpg
4,Stüssy,Women's Clothing,Stüssy / Nike Insulated Pullover Jacket,SOLD OUT ALL SALES FINAL \nStüssy and Nike re...,None,$350.00,[Black],None,https://www.stussy.com/collections/all-womens/...,https://cdn.shopify.com/s/files/1/0087/6193/39...,2020-12-11,stüssy-women_5.jpg


In [56]:
second_pass.to_csv('stüssy-women.csv', index = False, encoding = 'utf-8-sig')

In [57]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/stüssy-women.csv', encoding='utf-8-sig')

os.mkdir('stüssy-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('stüssy-women/'+ "stüssy-women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1